In [1]:
import pandas as pd
import pulp

In [2]:
# Create dictionaries for costs and constraints
inbound_dict = {'Stylish':{'Factory 1':{'Crossdock 1':30, 'Crossdock 2':50} ,
                'Factory 2':{'Crossdock 1':23, 'Crossdock 2':66},
                'Factory 3':{'Crossdock 1':35, 'Crossdock 2':14} ,
                'Factory 4':{'Crossdock 1':70, 'Crossdock 2':12} ,
                'Factory 5':{'Crossdock 1':65, 'Crossdock 2':70}},
                'Leisure':{'Factory 1':{'Crossdock 1':33, 'Crossdock 2':55} ,
                'Factory 2':{'Crossdock 1':25, 'Crossdock 2':73},
                'Factory 3':{'Crossdock 1':39, 'Crossdock 2':15} ,
                'Factory 4':{'Crossdock 1':77, 'Crossdock 2':13} ,
                'Factory 5':{'Crossdock 1':12, 'Crossdock 2':14}}
}

In [3]:
outbound_dict = {'Stylish':{'Crossdock 1':{'DC1':12, 'DC2':25, 'DC3':22, 'DC4':40, 'DC5':41} ,
                'Crossdock 2':{'DC1':65, 'DC2':22, 'DC3':23, 'DC4':12, 'DC5':15} } ,
                'Leisure':{'Crossdock 1':{'DC1':13, 'DC2':28, 'DC3':24, 'DC4':44, 'DC5':45} ,
                'Crossdock 2':{'DC1':72, 'DC2':24, 'DC3':25, 'DC4':13, 'DC5':17} } ,
}

In [4]:
demand_dict = {'Stylish':{'DC1':130, 'DC2':45, 'DC3':70, 'DC4':100, 'DC5':5} ,
                'Leisure':{'DC1':15, 'DC2':45, 'DC3':40, 'DC4':100, 'DC5':175}
}

In [5]:
capacity_dict = {'Stylish':{'Factory 1':150, 'Factory 2': 300, 'Factory 3':90, 'Factory 4':140, 'Factory 5':220} ,
            'Leisure':{'Factory 1':200, 'Factory 2': 300, 'Factory 3':70, 'Factory 4':30, 'Factory 5':220} ,
            'Combined':{'Factory 1':200, 'Factory 2': 300, 'Factory 3':100, 'Factory 4':150, 'Factory 5':220}
}

In [6]:
# Create list of factory, CDs, and DCs
factories = ['Factory 1', 'Factory 2', 'Factory 3', 'Factory 4', 'Factory 5']
CDs = ['Crossdock 1', 'Crossdock 2']
DCs = ['DC1', 'DC2', 'DC3', 'DC4', 'DC5']
Styles = ['Stylish', 'Leisure']  #, 'Combined'

In [7]:
# Model
model = pulp.LpProblem("Network Design", pulp.LpMinimize)

c:\Users\Local User\AppData\Local\Programs\Python\Python312\Lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [8]:
# Decision variables
inbound = pulp.LpVariable.dicts("Inbound", (Styles, factories, CDs), lowBound=0, cat='Integer')
outbound = pulp.LpVariable.dicts("Outbound", (Styles, CDs, DCs), lowBound=0, cat='Integer')


In [9]:
# Objective function: Minimize total cost
model += pulp.lpSum(inbound[Style][factory][CD] * inbound_dict[Style][factory][CD] 
                    for Style in Styles for factory in factories for CD in CDs) + \
         pulp.lpSum(outbound[Style][CD][DC] * outbound_dict[Style][CD][DC] 
                    for Style in Styles for CD in CDs for DC in DCs)

In [10]:
# Demand constraint
for Style in Styles:  
    for DC in DCs:
        constraint_name = f"Demand_Constraint_{Style}_{DC}"
        model += pulp.lpSum(outbound[Style][CD][DC] for CD in CDs) >= demand_dict[Style][DC], constraint_name


In [11]:
# Combined capacity constraint
for factory in factories:
    constraint_name = f"Capacity_Constraint_Combined_{factory}"
    model += pulp.lpSum(inbound[Style][factory][CD] for Style in Styles for CD in CDs) <= capacity_dict['Combined'][factory], constraint_name


In [12]:
# Conservation of flow constraint
for Style in Styles:
    for CD in CDs:
        constraint_name = f"CoF_Constraint_{Style}_{CD}"
        model += pulp.lpSum(inbound[Style][factory][CD] for factory in factories) == \
                 pulp.lpSum(outbound[Style][CD][DC] for DC in DCs), constraint_name


In [13]:
# Solve the model
model.solve()

1

In [14]:
# Output results
if pulp.LpStatus[model.status] == "Optimal":
    print(f"Status: Optimal")
    print(f"Total Cost: {pulp.value(model.objective)}\n")
    print("-" * 40)
    print("\nDecision Variable Values (non-zero):")
    for var in model.variables():
        if var.varValue != 0:
            print(f"{var.name} = {var.varValue}")
else:
    print(f"Status: {pulp.LpStatus[model.status]}")

Status: Optimal
Total Cost: 24230.0

----------------------------------------

Decision Variable Values (non-zero):
Inbound_Leisure_Factory_2_Crossdock_1 = 55.0
Inbound_Leisure_Factory_3_Crossdock_2 = 100.0
Inbound_Leisure_Factory_5_Crossdock_2 = 220.0
Inbound_Stylish_Factory_2_Crossdock_1 = 200.0
Inbound_Stylish_Factory_4_Crossdock_2 = 150.0
Outbound_Leisure_Crossdock_1_DC1 = 15.0
Outbound_Leisure_Crossdock_1_DC3 = 40.0
Outbound_Leisure_Crossdock_2_DC2 = 45.0
Outbound_Leisure_Crossdock_2_DC4 = 100.0
Outbound_Leisure_Crossdock_2_DC5 = 175.0
Outbound_Stylish_Crossdock_1_DC1 = 130.0
Outbound_Stylish_Crossdock_1_DC3 = 70.0
Outbound_Stylish_Crossdock_2_DC2 = 45.0
Outbound_Stylish_Crossdock_2_DC4 = 100.0
Outbound_Stylish_Crossdock_2_DC5 = 5.0
